# Exam Performance Trends Over Time

Interactive visualization of Portuguese national exam trends by year.

**Features:**
- **Dual Y-axes**: Average Grade (left, lines) and Number of Exams (right, bars)
- **Compare by**: All (aggregated), Gender, School Type, or NUTS2 Region
- **Bar modes**: Stacked or Dodged
- **Filters**: Gender, School Type, NUTS2, COVID period
- **Two-row legend**: Organized by metric type

In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_file, save, show
from bokeh.models import (
    ColumnDataSource, HoverTool, Select, CheckboxButtonGroup,
    CustomJS, Div, Range1d, NumeralTickFormatter, InlineStyleSheet,
    LinearAxis, Legend, LegendItem, RadioButtonGroup
)
from bokeh.layouts import column, row
from bokeh.io import output_notebook
from bokeh.palettes import Category10

output_notebook()

Loading BokehJS ...

## Load and Prepare Data

In [2]:
df = pd.read_csv('viz2_year_trends.csv')

genders = sorted(df['gender'].dropna().unique().tolist())
school_types = sorted(df['school_type'].dropna().unique().tolist())
nuts2_regions = sorted(df['nuts2'].dropna().unique().tolist())
years = sorted(df['year'].dropna().unique().tolist())
covid_periods = ['Before', 'After']

print(f"Loaded {len(df):,} rows")
print(f"Years: {years[0]} - {years[-1]}")
print(f"NUTS2 regions: {len(nuts2_regions)}")

Loaded 13,937 rows
Years: 2008 - 2024
NUTS2 regions: 8


In [3]:
df_agg = df.groupby(['year', 'gender', 'school_type', 'nuts2', 'covid_period']).agg({
    'num_exams': 'sum',
    'avg_grade': lambda x: np.average(x, weights=df.loc[x.index, 'num_exams']) if df.loc[x.index, 'num_exams'].sum() > 0 else 0
}).reset_index()

master_source = ColumnDataSource(df_agg)
print(f"Aggregated to {len(df_agg):,} rows")

Aggregated to 504 rows


## Define Colors and Create Data Sources

In [4]:
colors = {
    'All': '#2c3e50',  # Dark gray for "All"
    'F': '#e74c3c', 'M': '#3498db',
    'PRI': '#9b59b6', 'PUB': '#2ecc71',
}
for i, region in enumerate(nuts2_regions):
    colors[region] = Category10[10][i % 10]

In [5]:
def create_series_sources(df, series_col, series_values):
    sources = {}
    for val in series_values:
        subset = df[df[series_col] == val].groupby('year').agg({
            'num_exams': 'sum',
            'avg_grade': lambda x: np.average(x, weights=df.loc[x.index, 'num_exams']) if df.loc[x.index, 'num_exams'].sum() > 0 else 0
        }).reset_index().sort_values('year')
        sources[str(val)] = {
            'year': subset['year'].tolist(),
            'x': subset['year'].tolist(),
            'num_exams': subset['num_exams'].tolist(),
            'avg_grade': subset['avg_grade'].tolist(),
            'bottom': [0] * len(subset),
            'top': subset['num_exams'].tolist()
        }
    
    all_years = sorted(set(y for v in sources.values() for y in v['year']))
    cumulative = {y: 0 for y in all_years}
    for val in series_values:
        val_str = str(val)
        new_bottom, new_top = [], []
        for i, year in enumerate(sources[val_str]['year']):
            new_bottom.append(cumulative[year])
            new_top.append(cumulative[year] + sources[val_str]['num_exams'][i])
            cumulative[year] += sources[val_str]['num_exams'][i]
        sources[val_str]['bottom'] = new_bottom
        sources[val_str]['top'] = new_top
    return {k: ColumnDataSource(data=v) for k, v in sources.items()}

sources_gender = create_series_sources(df_agg, 'gender', genders)
sources_school = create_series_sources(df_agg, 'school_type', school_types)
sources_nuts2 = create_series_sources(df_agg, 'nuts2', nuts2_regions)

# Create "All" source - aggregated across all dimensions
df_all = df_agg.groupby('year').agg({
    'num_exams': 'sum',
    'avg_grade': lambda x: np.average(x, weights=df_agg.loc[x.index, 'num_exams']) if df_agg.loc[x.index, 'num_exams'].sum() > 0 else 0
}).reset_index().sort_values('year')

source_all = ColumnDataSource(data={
    'year': df_all['year'].tolist(),
    'x': df_all['year'].tolist(),
    'num_exams': df_all['num_exams'].tolist(),
    'avg_grade': df_all['avg_grade'].tolist(),
    'bottom': [0] * len(df_all),
    'top': df_all['num_exams'].tolist()
})

sources_flat = {'all_All': source_all}
for g in genders:
    sources_flat[f'gender_{g}'] = sources_gender[g]
for s in school_types:
    sources_flat[f'school_type_{s}'] = sources_school[s]
for n in nuts2_regions:
    sources_flat[f'nuts2_{n}'] = sources_nuts2[n]

## Create Figure with Dual Y-Axes

In [6]:
p = figure(
    width=950, height=580,
    title="Exam Performance Trends Over Time",
    x_axis_label="Year", y_axis_label="Average Grade",
    toolbar_location="above",
    tools="pan,wheel_zoom,box_zoom,reset,save",
    y_range=Range1d(start=0, end=20)
)

max_exams = df_agg.groupby('year')['num_exams'].sum().max()
p.extra_y_ranges['exams'] = Range1d(start=0, end=max_exams * 1.1)
p.add_layout(LinearAxis(
    axis_label="Number of Exams", y_range_name="exams",
    formatter=NumeralTickFormatter(format='0,0')
), 'right')

p.title.text_font_size = '16pt'
p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'
p.xaxis.axis_label_text_font_size = '13pt'
p.yaxis.axis_label_text_font_size = '13pt'
p.x_range.start = years[0] - 0.5
p.x_range.end = years[-1] + 0.5
p.yaxis[0].formatter = NumeralTickFormatter(format='0.0')

## Create Renderers

In [7]:
renderers_flat = {}
BAR_WIDTH, BAR_ALPHA, LINE_WIDTH = 0.7, 0.35, 2.5

def create_renderers(series_type, series_values, visible):
    for val in series_values:
        key = f'{series_type}_{val}'
        source = sources_flat[key]
        renderers_flat[f'{key}_bar'] = p.vbar(
            x='x', bottom='bottom', top='top', source=source,
            width=BAR_WIDTH, color=colors[val], alpha=BAR_ALPHA,
            y_range_name='exams', visible=visible
        )
        renderers_flat[f'{key}_line'] = p.line(
            'year', 'avg_grade', source=source,
            line_width=LINE_WIDTH, color=colors[val], visible=visible
        )
        renderers_flat[f'{key}_circle'] = p.scatter(
            'year', 'avg_grade', source=source,
            size=8, color=colors[val], visible=visible
        )

# Create "All" renderers (hidden by default)
create_renderers('all', ['All'], False)

# Create series renderers
create_renderers('gender', genders, True)
create_renderers('school_type', school_types, False)
create_renderers('nuts2', nuts2_regions, False)

p.add_tools(HoverTool(tooltips=[
    ('Year', '@year'), ('Avg Grade', '@avg_grade{0.2f}'), ('Num Exams', '@num_exams{0,0}')
]))

## Create Two-Row Legend

In [8]:
line_legend_items, bar_legend_items = {}, {}

def create_legend_items(series_type, series_values, visible):
    for val in series_values:
        key = f'{series_type}_{val}'
        line_legend_items[key] = LegendItem(
            label=val,
            renderers=[renderers_flat[f'{key}_line'], renderers_flat[f'{key}_circle']]
        )
        bar_legend_items[key] = LegendItem(
            label=val,
            renderers=[renderers_flat[f'{key}_bar']]
        )
        line_legend_items[key].visible = visible
        bar_legend_items[key].visible = visible

# Create "All" legend items (hidden by default)
create_legend_items('all', ['All'], False)

# Create series legend items
create_legend_items('gender', genders, True)
create_legend_items('school_type', school_types, False)
create_legend_items('nuts2', nuts2_regions, False)

In [9]:
all_line_items = [line_legend_items['all_All']] + \
                 [line_legend_items[f'gender_{g}'] for g in genders] + \
                 [line_legend_items[f'school_type_{s}'] for s in school_types] + \
                 [line_legend_items[f'nuts2_{n}'] for n in nuts2_regions]

all_bar_items = [bar_legend_items['all_All']] + \
                [bar_legend_items[f'gender_{g}'] for g in genders] + \
                [bar_legend_items[f'school_type_{s}'] for s in school_types] + \
                [bar_legend_items[f'nuts2_{n}'] for n in nuts2_regions]

legend_lines = Legend(
    items=all_line_items, orientation="horizontal", location="top_left",
    title="Average Grade:", title_text_font_style="bold",
    title_text_font_size="10pt", label_text_font_size="9pt",
    spacing=15, padding=5, margin=0, click_policy="hide"
)
legend_bars = Legend(
    items=all_bar_items, orientation="horizontal", location="top_left",
    title="Number of Exams:", title_text_font_style="bold",
    title_text_font_size="10pt", label_text_font_size="9pt",
    spacing=15, padding=5, margin=0, click_policy="hide"
)

p.add_layout(legend_lines, 'below')
p.add_layout(legend_bars, 'below')

## Create Controls

In [10]:
series_select = Select(
    title="", value="gender", width=150,
    options=[("all", "All"), ("gender", "Gender"), ("school_type", "School Type"), ("nuts2", "NUTS2 Region")]
)
bar_mode_toggle = RadioButtonGroup(labels=["Stacked", "Dodged"], active=0, width=150)

BUTTON_CSS = """
:host(.bk-CheckboxButtonGroup) .bk-btn, :host(.bk-RadioButtonGroup) .bk-btn {
    background-color: #e6e6e6 !important; border-color: #ccc !important; color: #333 !important;
}
:host(.bk-CheckboxButtonGroup) .bk-btn.bk-active, :host(.bk-RadioButtonGroup) .bk-btn.bk-active {
    background-color: #3071a9 !important; border-color: #285e8e !important; color: white !important;
}
"""
button_stylesheet = InlineStyleSheet(css=BUTTON_CSS)
bar_mode_toggle.stylesheets = [button_stylesheet]

DIMMED = {'opacity': '0.3', 'pointer-events': 'none'}
ACTIVE = {'opacity': '1', 'pointer-events': 'auto'}

gender_filter = CheckboxButtonGroup(labels=genders, active=list(range(len(genders))), stylesheets=[button_stylesheet], styles=DIMMED)
school_filter = CheckboxButtonGroup(labels=school_types, active=list(range(len(school_types))), stylesheets=[button_stylesheet], styles=ACTIVE)
nuts2_filter = CheckboxButtonGroup(labels=nuts2_regions, active=list(range(len(nuts2_regions))), stylesheets=[button_stylesheet], styles=ACTIVE)
covid_filter = CheckboxButtonGroup(labels=covid_periods, active=list(range(len(covid_periods))), stylesheets=[button_stylesheet])

In [11]:
LABEL = {'display': 'flex', 'align-items': 'center', 'height': '30px'}
LABEL_DIM = {'display': 'flex', 'align-items': 'center', 'height': '30px', 'opacity': '0.3', 'pointer-events': 'none'}

series_label = Div(text="<b>Compare by:</b>", width=80, styles=LABEL)
barmode_label = Div(text="<b>Bar mode:</b>", width=80, styles=LABEL)
gender_label = Div(text="<b>Gender:</b>", width=80, styles=LABEL_DIM)
school_label = Div(text="<b>School:</b>", width=80, styles=LABEL)
nuts2_label = Div(text="<b>NUTS2:</b>", width=80, styles=LABEL)
covid_label = Div(text="<b>COVID:</b>", width=80, styles=LABEL)
filter_status = Div(text="", width=900, styles={'color': '#e67e22', 'font-size': '12px', 'font-style': 'italic', 'margin': '5px 0'})

## JavaScript Callback

In [12]:
js_code = """
const series_type = series_select.value;
const is_stacked = bar_mode_toggle.active === 0;

const DIMMED = {'opacity': '0.3', 'pointer-events': 'none'};
const ACTIVE = {'opacity': '1', 'pointer-events': 'auto'};
const LABEL_DIM = {'display': 'flex', 'align-items': 'center', 'height': '30px', 'opacity': '0.3', 'pointer-events': 'none'};
const LABEL_ACT = {'display': 'flex', 'align-items': 'center', 'height': '30px', 'opacity': '1', 'pointer-events': 'auto'};

const series_map = {'all': ['All'], 'gender': genders_list, 'school_type': schools_list, 'nuts2': nuts2_list};
const covid_all = ['Before', 'After'];

const genders_sel = gender_filter.active.map(i => genders_list[i]);
const schools_sel = school_filter.active.map(i => schools_list[i]);
const nuts2_sel = nuts2_filter.active.map(i => nuts2_list[i]);
const covid_sel = covid_filter.active.map(i => covid_all[i]);

const data = master_source.data;
const n = data['year'].length;

function filterAndAggregate(series_col, series_values) {
    const result = {};
    for (const sv of series_values) result[sv] = {};
    
    for (let i = 0; i < n; i++) {
        // For "all", apply all filters; for others, skip the series dimension filter
        if (series_col !== 'gender' && series_col !== 'all' && !genders_sel.includes(data['gender'][i])) continue;
        if (series_col === 'all' && !genders_sel.includes(data['gender'][i])) continue;
        if (series_col !== 'school_type' && series_col !== 'all' && !schools_sel.includes(data['school_type'][i])) continue;
        if (series_col === 'all' && !schools_sel.includes(data['school_type'][i])) continue;
        if (series_col !== 'nuts2' && series_col !== 'all' && !nuts2_sel.includes(data['nuts2'][i])) continue;
        if (series_col === 'all' && !nuts2_sel.includes(data['nuts2'][i])) continue;
        if (!covid_sel.includes(data['covid_period'][i])) continue;
        
        // For "all", aggregate everything into single "All" key
        const sv = series_col === 'all' ? 'All' : String(data[series_col][i]);
        if (!series_values.includes(sv)) continue;
        
        const yr = data['year'][i], num = data['num_exams'][i];
        if (!result[sv][yr]) result[sv][yr] = {num_exams: 0, grade_sum: 0};
        result[sv][yr].num_exams += num;
        result[sv][yr].grade_sum += data['avg_grade'][i] * num;
    }
    for (const sv in result)
        for (const yr in result[sv]) {
            const t = result[sv][yr].num_exams;
            result[sv][yr].avg_grade = t > 0 ? result[sv][yr].grade_sum / t : 0;
        }
    return result;
}

const cur_vals = series_map[series_type];
const agg = filterAndAggregate(series_type, cur_vals);
const all_yrs = [...new Set(Object.values(agg).flatMap(o => Object.keys(o).map(Number)))].sort((a,b) => a-b);

const n_ser = cur_vals.length, tw = 0.7;
const bw = is_stacked ? tw : tw / n_ser;
const off0 = is_stacked ? 0 : -tw/2 + bw/2;

const cum = {}; all_yrs.forEach(y => cum[y] = 0);
let max_g = 0, max_e_st = 0, max_e_si = 0;

// Hide all renderers and legend items
for (const st in series_map)
    for (const sv of series_map[st]) {
        const k = st + '_' + sv;
        if (all_renderers[k+'_bar']) all_renderers[k+'_bar'].visible = false;
        if (all_renderers[k+'_line']) all_renderers[k+'_line'].visible = false;
        if (all_renderers[k+'_circle']) all_renderers[k+'_circle'].visible = false;
    }
for (const k in line_items) { line_items[k].visible = false; bar_items[k].visible = false; }

let si = 0;
for (const sv of cur_vals) {
    const k = series_type + '_' + sv, src = all_sources[k];
    const xo = is_stacked ? 0 : off0 + si * bw;
    const yrs=[], xs=[], nums=[], avgs=[], bots=[], tops=[];
    for (const yr of all_yrs) {
        const d = agg[sv][yr];
        if (d) {
            yrs.push(yr); xs.push(yr + xo); nums.push(d.num_exams); avgs.push(d.avg_grade);
            if (is_stacked) { bots.push(cum[yr]); tops.push(cum[yr] + d.num_exams); cum[yr] += d.num_exams; }
            else { bots.push(0); tops.push(d.num_exams); }
            if (d.avg_grade > max_g) max_g = d.avg_grade;
            if (d.num_exams > max_e_si) max_e_si = d.num_exams;
        }
    }
    src.data = {year: yrs, x: xs, num_exams: nums, avg_grade: avgs, bottom: bots, top: tops};
    src.change.emit();
    all_renderers[k+'_bar'].glyph.width = bw;
    if (yrs.length) {
        all_renderers[k+'_bar'].visible = true;
        all_renderers[k+'_line'].visible = true;
        all_renderers[k+'_circle'].visible = true;
        line_items[k].visible = true; bar_items[k].visible = true;
    }
    si++;
}

for (const yr in cum) if (cum[yr] > max_e_st) max_e_st = cum[yr];
const max_e = is_stacked ? max_e_st : max_e_si;

if (max_g > 0) { plot.y_range.start = 0; plot.y_range.end = Math.min(20, max_g * 1.15); }
if (max_e > 0) { plot.extra_y_ranges['exams'].start = 0; plot.extra_y_ranges['exams'].end = max_e * 1.1; }

// Update filter opacity - all filters active when "all" is selected
gender_filter.styles = (series_type === 'gender') ? DIMMED : ACTIVE;
gender_label.styles = (series_type === 'gender') ? LABEL_DIM : LABEL_ACT;
school_filter.styles = (series_type === 'school_type') ? DIMMED : ACTIVE;
school_label.styles = (series_type === 'school_type') ? LABEL_DIM : LABEL_ACT;
nuts2_filter.styles = (series_type === 'nuts2') ? DIMMED : ACTIVE;
nuts2_label.styles = (series_type === 'nuts2') ? LABEL_DIM : LABEL_ACT;

const excl = [];
if (series_type !== 'gender' && genders_sel.length < genders_list.length)
    excl.push('Gender: ' + genders_list.filter(g => !genders_sel.includes(g)).join(', '));
if (series_type !== 'school_type' && schools_sel.length < schools_list.length)
    excl.push('School: ' + schools_list.filter(s => !schools_sel.includes(s)).join(', '));
if (series_type !== 'nuts2' && nuts2_sel.length < nuts2_list.length) {
    const ex = nuts2_list.filter(n => !nuts2_sel.includes(n));
    excl.push('NUTS2: ' + (ex.length <= 3 ? ex.join(', ') : ex.length + ' regions'));
}
if (covid_sel.length < covid_all.length)
    excl.push('COVID: ' + covid_all.filter(c => !covid_sel.includes(c)).join(', '));
filter_status.text = excl.length ? 'Excluding: ' + excl.join(' | ') : '';
"""

In [13]:
callback = CustomJS(
    args=dict(
        plot=p, master_source=master_source,
        series_select=series_select, bar_mode_toggle=bar_mode_toggle,
        gender_filter=gender_filter, school_filter=school_filter,
        nuts2_filter=nuts2_filter, covid_filter=covid_filter,
        gender_label=gender_label, school_label=school_label, nuts2_label=nuts2_label,
        filter_status=filter_status,
        all_renderers=renderers_flat, all_sources=sources_flat,
        line_items=line_legend_items, bar_items=bar_legend_items,
        genders_list=genders, schools_list=school_types, nuts2_list=nuts2_regions
    ),
    code=js_code
)

for ctrl in [series_select, bar_mode_toggle, gender_filter, school_filter, nuts2_filter, covid_filter]:
    ctrl.js_on_change('active' if hasattr(ctrl, 'active') and not isinstance(ctrl, Select) else 'value', callback)

## Assemble Layout

In [14]:
layout = column(
    row(series_label, series_select, Div(width=30), barmode_label, bar_mode_toggle,
        sizing_mode="stretch_width",
        styles={'border-bottom': '1px solid #ccc', 'padding-bottom': '8px', 'margin-bottom': '8px'}),
    row(gender_label, gender_filter, Div(width=20),
        school_label, school_filter, Div(width=20),
        covid_label, covid_filter),
    row(nuts2_label, nuts2_filter),
    row(filter_status),
    row(p),
    sizing_mode="fixed",
    styles={'border': '1px solid #ccc', 'padding': '10px'}
)

show(layout)

## Export to HTML

In [15]:
output_file("./../site/viz2.html", title="Exam Performance Trends Over Time")
save(layout)
print("Saved to viz2.html")

Saved to viz2.html
